# EEGLAB Tutorial #2: Using an Existing EEGLAB Montage

**Author**: Katie Douglas  
**Date**: 2022-01-13

## Overview

In this tutorial, we demonstrate how to import epoched EEGLAB data into HyPyP using an existing EEGLAB montage file. Instead of creating a montage from scratch, we load channel locations from a pre‐formatted EEGLAB channel locations file (e.g., a `.locs` file).

(See https://eeglab.org/tutorials/04_Import/Channel_Locations.html#supported-data-formats for more information)

This notebook covers:

1. Loading epoched data and raw data from EEGLAB
2. Equalizing the number of epochs between participants
3. Loading and creating a custom montage from an EEGLAB montage file
4. Applying the montage to both raw and epoched data
5. Visualizing sensor locations and dropping EOG channels

The channel location visualtizations in this tutorial use MNE visualizations. Please note that MNE and EEGLAB channel topomap and channel location visualizations are known to be slightly different (MNE provides a detailed discussion about this here https://mne.tools/stable/auto_examples/visualization/eeglab_head_sphere.html).

## 1. Load in the Epoched Data

For each participant, both the `.set` and `.fdt` files should be stored in the same directory. In this example, we use the sample dataset provided by EEGLAB. Although participants do not need to have the same number of epochs, they must share the same sampling rate (i.e. the same number of samples per epoch).

In [ ]:
import mne
import warnings

# Suppress RuntimeWarnings for cleaner output
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Define file paths for the epoched data for two participants
path_1 = "../data/EEGLAB/eeglab_data_epochs_ica.set"
path_2 = "../data/EEGLAB/eeglab_data_epochs_ica.set"

# Load the epoched data using MNE's EEGLAB reader
epo1 = mne.io.read_epochs_eeglab(path_1)
epo2 = mne.io.read_epochs_eeglab(path_2)

print('Epoched data loaded for both participants.')

## 2. Read in the Raw Data

Load the raw EEG data (not epoched) from EEGLAB. This raw data will be used to apply the montage and inspect channel locations.

In [ ]:
# Define the file path for the raw EEGLAB data
raw_data_path = "../data/EEGLAB/eeglab_data.set"

# Read the raw data using MNE
raw_data = mne.io.read_raw_eeglab(raw_data_path)

print('Raw data loaded.')

## 3. Equalize the Number of Epochs

Ensure that both participants have the same number of epochs (if possible) for further analysis.

In [ ]:
mne.epochs.equalize_epoch_counts([epo1, epo2])
print('Epoch counts equalized between participants.')

## 4. Load in the EEGLAB Montage

Specify the path to the EEGLAB montage file. In this example, we use a `.locs` file containing the channel locations for a 32-channel montage.

In [ ]:
eeglab_montage = "../data/EEGLAB/eeglab_chan32.locs"
print('EEGLAB montage file path set.')

## 5. Create a Custom EEGLAB Montage

Use MNE's `read_custom_montage` function to create a montage from the provided file.

In [ ]:
montage = mne.channels.read_custom_montage(eeglab_montage)
print('Custom EEGLAB montage created.')

## 6. Set the Custom EEGLAB Montage on Raw Data and Visualize

Apply the montage to the raw data and plot the sensor locations. Note that two channels may appear off the head – these are typically the EOG channels.

In [ ]:
# Apply the custom montage to the raw data
raw_data.set_montage(montage)

# Plot sensor locations using MNE's visualization
raw_data.plot_sensors()

print('Montage applied to raw data and sensors plotted.')

### Inspect Channel Names and Information

Print the channel names and raw data information to check for channels that are not located on the head (e.g., EOG channels).

In [ ]:
print(raw_data.info['ch_names'])

# Display the full info structure
raw_data.info

As MNE recognizes these EEGLAB EOG channels as EEG channels, the EOG channels will need to dropped manually.

## 7. Apply the Custom Montage to the Epoched Data

Set the montage for the epoched data and plot the sensor layout. At this stage, EOG channels are still present.

In [ ]:
epo1.set_montage(montage)
epo1.plot_sensors()

print('Montage applied to epoched data (epo1).')

## 8. Drop the EOG Channels

Since MNE recognizes the EOG channels as EEG channels, we need to drop them manually. First, we drop them from `epo1` and then plot the sensor locations to verify that the EOG channels are removed.

In [ ]:
epo1.drop_channels(['EOG1', 'EOG2'])
epo1.plot_sensors()

print('EOG channels dropped from epo1.')

## 9. Apply the Montage and Drop EOG Channels in One Step for `epo2`

For convenience, you can set the montage and drop the EOG channels in a single workflow. The following cell demonstrates this for `epo2`.

In [ ]:
epo2.set_montage(montage)
epo2.drop_channels(['EOG1', 'EOG2'])
epo2.plot_sensors()

print('Montage applied and EOG channels dropped for epo2.')